## Monte Carlo Simulation for the modified network case33bw from matpower 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('..\matplot.mplstyle')
import pandapower as pp
import openpyxl
from datetime import datetime

In [2]:
# Number of buses
nbus = 42
# Numeber of samples
N = 1000
# timesteps of 10min 
time = 144
# Residential loadshape with 144 pts
k = np.array([0.032740281, 0.019707864, 0.041622676, 0.130779404, 0.139621404, 0.130085024, 0.070994622, 0.070728025,
     0.120049613, 0.117644786, 0.088727972, 0.0657238, 0.075095686, 0.074816533, 0.084021978, 0.063546442,
     0.084713633, 0.098523066, 0.041773349, 0.048624678, 0.048124771, 0.013152819, 0,0.009682771, 0.023338222,
     0.073688823, 0.114733628, 0.065458371, 0.132573364, 0.097184336, 0.109735925, 0.047572694, 0.065406979,
     0.089092779, 0.069054662, 0.056740471, 0.062022486, 0.141774332, 0.116577618, 0.121112012, 0.182448392,
     0.174978896, 0.260531351, 0.37397735, 0.227811218, 0.336294359, 0.35831468, 0.408185426, 0.491406346,
     0.482170922, 0.520398204, 0.574318038, 0.603267847, 0.737778252, 0.721634461, 0.590997457, 0.653740019,
     0.563592822, 0.593654963, 0.564458704, 0.569416695, 0.650488484, 0.711874505, 0.662122215, 0.620664228,
     0.633792032, 0.618355274, 0.525782713, 0.587935626, 0.502503572, 0.541578924, 0.562174862, 0.575472806,
     0.593025602, 0.534658585, 0.574419168, 0.622300896, 0.542334138, 0.645609139, 0.63763594, 0.553571816,
     0.545721842, 0.523726438, 0.495058702, 0.467341722, 0.428146358, 0.433008962, 0.440977586, 0.527905856,
     0.607364629, 0.636883744, 0.708781138, 0.62784289, 0.672351125, 0.502472231, 0.527957929, 0.565853108,
     0.530876679, 0.447205103, 0.394676851, 0.462093384, 0.609759528, 0.636822035, 0.631154285, 0.766115345,
     0.790480634, 0.842917586, 0.768005082, 0.794547047, 0.73209123, 0.740249071, 0.754541378, 0.853188253,
     0.792092287, 0.824462408, 0.878812262, 0.846392307, 1,0.949467968, 0.986766783, 0.982374983, 0.941293775,
     0.801338613, 0.763749257, 0.636801984, 0.606767486, 0.640203217, 0.576829738, 0.550041128, 0.58568371,
     0.476521082, 0.550973976, 0.417836465, 0.47379846, 0.478021775, 0.354373536, 0.342623492, 0.317807158,
     0.232000077, 0.232305803, 0.213533901, 0.155210295, 0.127120235, 0.068917713])
# Loads power factor
fp = [ 0, 0.8575, 0.9138, 0.8321, 0.8944, 0.9487, 0.8944, 0.8944, 0.9487, 0.9487, 0.8321, 0.8638, 0.8638,
       0.8321, 0.9864, 0.9487, 0.9487, 0.9138, 0.9138, 0.9138, 0.9138, 0.9138, 0.8742, 0.9029, 0, 0,
       1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9231, 0.9231, 0.9487, 0.8638, 0.3162,
       0.9062, 0.9029, 0.8321]
# pv loadshape 1000 x 144
pv = np.load(r'../dataset/PV_Curves_10000_Samples.npz')
pv = pv['PV_w']

In [ ]:
# # Loadshape Figure
# net = pp.from_excel(r"case33bw_mod.xlsx")
# mi_Pc = np.sum(net.load["p_mw"].values * k[:, None], axis = 1)
# t = np.arange(time)
# plt.figure(figsize = (13,6), dpi = 300)
# plt.plot(t,mi_Pc)
# plt.show
# plt.xticks(np.arange(0, 145, 12), ('0h', '2h', '4h', '6h', '8h', '10h', '12h', '14h','16h', '18h', '20h', '22h', '24h'))
# plt.ylabel('MW')
# plt.ylim(top = 5)
# plt.title(f'Loadshape: sum of loads demand in every timestep', fontsize=22)
# plt.grid(True)
# plt.savefig(f"LoadShape_K.svg")
# plt.plot()

In [6]:
# Initialize the arrays for the P and Q load demands, Bus voltages 
# and Active power of transformers at buses 1 and 25 (in this file 0 and 24)
demand_p = np.zeros((time*N, nbus))
demand_q = np.zeros((time*N, nbus))
voltages = np.zeros((time*N, nbus))
p_trafo1 = np.zeros((time*N, 1))
p_trafo2 = np.zeros((time*N, 1))

# Initialize a random generator
rng = np.random.default_rng()

# Loop throught the timesteps
for t in range(time):
    
    # Reload the network at every timestep to always use the original model
    net = pp.from_excel(r"case33bw_mod.xlsx")
    
    # Generate mean and std for the normal distribution
    mi_Pc = net.load["p_mw"] * k[t]
    std = mi_Pc * 0.5
    # Create the loadshape for every bus in the network
    pc = np.zeros((42, N))
    qc = np.zeros((42, N))
    for b in range(nbus):
        pc[b] = rng.normal(mi_Pc[b],std[b],N)
        qc[b] = pc[b]*np.tan(np.arccos(fp[b]))
    
    # Loop in every Monte Carlo sample for every timestep
    for n in range(N):
        # Update the network loads with the generated loadshapes
        net.load["p_mw"] = pc[:, n]
        net.load["q_mvar"] = qc[:, n]
        # Update the loads where the pv are connected using the data generated early
        net.load.loc[[11, 17, 24], "p_mw"] = net.load.loc[[11, 17, 24], "p_mw"] - pv[n, t]
        net.load.loc[[28], "p_mw"] = net.load.loc[[28], "p_mw"] - (1/47)*pv[n, t]
        net.load.loc[[31], "p_mw"] = net.load.loc[[31], "p_mw"] - (1/42)*pv[n, t]
        
        # Run the power flow
        results = pp.runpp(net, numba=False)
        
        # Collect the results ready to be used by the Neural Network
        # Demand_P,Q and Voltages have the shape of N*144 x 42
        # Transformers_P,Q have the shape of N*144 x 1
        demand_p[t + (time*n), :] = pc[:, n]
        demand_q[t + (time*n), :] = qc[:, n]
        voltages[t + (time*n), :] = net.res_bus["vm_pu"]
        p_trafo1[t + (time*n)] = net.res_line.loc[0, "p_from_mw"]
        p_trafo2[t + (time*n)] = net.res_line.loc[24, "q_from_mvar"]


# Save all this data in a numpy file format
date = datetime.now().strftime('%d_%m-%H_%M')
np.savez(fr"../dataset/MC_Simulation_std_05_samples_{N}_{date}", demand_p=demand_p, demand_q=demand_q, voltages=voltages, p_trafo1=p_trafo1, p_trafo2=p_trafo2)
# To load the files use data = np.load(filename) and data['column']